# Feed-forward сети

In [1]:
import os
os.chdir("./GB-Pytorch")

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))
                              ])
BATCH_SIZE = 10

In [4]:
train_set = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

Files already downloaded and verified


In [5]:
test_set = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified


In [6]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
class CustomLeakyRelu(nn.Module):
    def __init__(self, border, alpha):
        super().__init__()
        self.border = border
        self.alpha = alpha

    def forward(self, input):
        x = F.leaky_relu(input)
        return torch.where(x > self.border, x*self.alpha, x)

In [8]:
class FFNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        img_vector_size = 3072
        self.fc1 = nn.Linear(img_vector_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 100)
        self.cust_relu = CustomLeakyRelu(0.1, 0.5)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.cust_relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def predict(self, x):
        x = x.view(x.shape[0], -1)

        x = self.cust_relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [9]:
ff_network = FFNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ff_network.parameters(), lr=0.001)

In [18]:
EPOCHS = 20
ff_network.train()

for epoch in range(EPOCHS):
    loss_value = 0
    for batch_index, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        pred = ff_network(images)

        loss = criterion(pred, labels)
        loss_value += loss.item()

        loss.backward()
        optimizer.step()

    loss_value = loss_value / len(train_loader.dataset)
    print(f'Epoch: {epoch + 1}, loss: {loss_value}')


Epoch: 1, loss: 0.3145322729802132
Epoch: 2, loss: 0.30961189719200133
Epoch: 3, loss: 0.30552163642168045
Epoch: 4, loss: 0.30142009315013885
Epoch: 5, loss: 0.29818970482587814
Epoch: 6, loss: 0.2947867235565186
Epoch: 7, loss: 0.29189574337482455
Epoch: 8, loss: 0.2884645274114609
Epoch: 9, loss: 0.2855368833374977
Epoch: 10, loss: 0.2836717459464073
Epoch: 11, loss: 0.28131931375265123
Epoch: 12, loss: 0.27899888825654984
Epoch: 13, loss: 0.27651047065019607
Epoch: 14, loss: 0.27503158229589464
Epoch: 15, loss: 0.27246819865345956
Epoch: 16, loss: 0.27179698873281477
Epoch: 17, loss: 0.26979806173563003
Epoch: 18, loss: 0.267943428299427
Epoch: 19, loss: 0.2664293702864647
Epoch: 20, loss: 0.265063032143116
